<a href="https://colab.research.google.com/github/a-saadallah/geemap/blob/master/Tz_Rice_Indices_Outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This notebook will generate and evaluate the robustness of various remote-sensing based indices to predict yields, using the following steps:

1. Identify the cropland pixels in the study area:
     a. Mask out non-crop pixels in the study area 
     b. Generate a (or multiple) crop layers -- focused on rice as the key crop      of interest

     Our starting bounding box will include a region surrounding 
     4.21 South,	38.06 E
     4.24 South, 38.02 E


2. Extract (and visualize a sub-sample of) the following vegetative indices for the crop layer(s) over the time period (with line graphs showing the growing season, e.g. October - March, from 2003-2012).
    a. NDVI
    b. EVI
    c. LAI
    d. fPAR
    e. Cumulative Rainfall (CHIRPS)
    f. GPP

3. Develop relevant metrics to summarize the indices for each year:
    a. AUC for growing season
    b. Mean
    c. Median
    d. Maximum
    
    **we may also "normalize" the values (divide the annual value "y" by the regional mean y_z over the observed time period) to aid comparisons 




# Header

### Header Info

* *Authors:* EB & AS
* *Date started*: January 2020
* *Purpose*: extract/munge indices in each zone
* *Inputs*: sample zone area
* *Outputs*: 
* *Notes*: Style guide here: https://stmorse.github.io/journal/tidyverse-style-pandas.html



## Setup

In [1]:
# install autopep
#!pip install autopep8

### 1. Import Python EE API and relevant modules

In [2]:
# Import modules
import ee
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from math import pi
import numpy as np
from datetime import datetime
import itertools
import folium
#import autopep8

### 2. Initiate EE authentication

In [3]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=A3oLvQPAtZjrkUXi6tguhb-DS9zYz_8qH4KcDwKuijQ&tc=-5W-wQs4IxaFyFXDF2Kb81s53991DRa-u4UvLut7k0Q&cc=ZU7xEaU790Q9zx74Rxgp4n6kzOYGJaUecJ1MjB2p9qM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhd9UvWngphjhEBAn4ybtbiHJA2PX74MpYN_F1J1Xhr298vLTFfI0k

Successfully saved authorization token.


### 3. Extract MODIS data and generate variables
In this code block we load our Modis image collection and filter the collection based on date range. 


In [4]:
# Region of interest
poly = ee.Geometry.Rectangle([[38.06, -4.21], [38.02, -4.24]])

# Filter MODIS image collection from 2001 to 2013
modis_coll = ee.ImageCollection('MODIS/006/MOD13Q1').filter(
    ee.Filter.calendarRange(2001, 2013, 'year'))

### 4. Harmonic Analysis
Here we build up Harmonic analsyis function. Harmonic analysis, also known as Fourier Analysis or spectral analysis, is considered an efficient models for reconstructing and smoothing vegetation indicies time series with periodicity to identify and characterize phenological parameters of natural as well as agricultural vegetation types. It decomposes a time-dependent periodic phenomenon into a series of sinusoidal waves, each with a unique amplitude $A$ and phase value $\phi$ (https://www.asprs.org/wp-content/uploads/pers/2001journal/april/2001_apr_461-470.pdf)

The formula used in this script:

$$ NDVI_t=  β_0 + β_{1t}+ \Sigma_{k=1}^{q} (U{k1} cos(2πωkt)+U_{k2}sin(2πω_kt))$$

See more in the book on Time Serieas Analysis from https://www.stat.pitt.edu/stoffer/tsa4/tsa4.pdf #pg. 169 equation 4.4

$β_0$ is a constant and $\beta_{1t}$ is the linear term, $U_{k1}$, $U_{k2}$, for k = 1, 2, . . ., q, are independent zero-mean random variables, $ω$ is the frequency index (cycle per unit time) and $A$ reflects the height or amplitude of the function,




In [5]:
# Prepare harmonic analysis components
dependent = ee.String('NDVI')  # The dependant variable
harmonics = 3  # Number of harmonic terms
harmonicFrequencies = ee.List.sequence(1, harmonics)
# Names list for cosine waves bands
cosNames = ee.List(['cos_' + str(elem) for elem in list(range(harmonics))])
# Names list for sine waves bands
sinNames = ee.List(['sin_' + str(elem) for elem in list(range(harmonics))])
independents = ee.List(['constant', 't']).cat(cosNames.cat(
    sinNames))  # list of all the independend bands names


# Compute harmonics for each image in the collection
def harmonic_var(img):
    # system time_start is a property of each image (image metadata) which store the date and time of image capture (in millisecond from 1970)
    date = img.get('system:time_start')
    years = ee.Date(date).difference(ee.Date('1970-01-01'), 'year')
    # turn date value into a band that represent time in radians
    timeRadians = ee.Image(years.multiply(2 * pi)).float().rename('t')
    constant = ee.Image(1).rename('constant')
    # scale NDVI and EVI values to be between 0 and 1
    ndvi = img.select('NDVI').divide(10000)
    evi = img.select('EVI').divide(10000)
    frequencies = ee.Image.constant(harmonicFrequencies)
    cosines = timeRadians.multiply(frequencies).cos().rename(
        cosNames)  # Get the cosine terms
    sines = timeRadians.multiply(frequencies).sin().rename(
        sinNames)  # Get the sine terms
    image = img.select([]).addBands(
        [ndvi, evi, timeRadians, constant, cosines, sines])  # Selecting bands we need for furthure processing
    return image.clip(poly)  # clip the image bands to the extent of study area


# applying the 'harmonic_var' function to our MODIS image collection
harmonic_modis = modis_coll.map(harmonic_var)

### 5. Apply yearly Harmonic Fit
After computing the harmonic independant variables, a harmonic fit will be applied on yearly basis. In order to do that, we create a list ranging from 2002 to 2013 and use its values to filter our image collection.

In [6]:
# Creating an empty image collection which will be filled with images on yearly basis inside the for loop
fittedHarmonic = ee.ImageCollection([])

for year in range(2001, 2014):
    year_coll = harmonic_modis.filter(
        # filter image colection by year
        ee.Filter.calendarRange(year, year, 'year')).merge(
        harmonic_modis.filter(
            ee.Filter.calendarRange(year - 1, year - 1, 'year')))

    harmonicTrend = year_coll.select(independents.add(dependent)).reduce(
        ee.Reducer.linearRegression(independents.length(), 1))  # Apply a linear regression to compute coefficients
    harmonicTrendCoeffs = harmonicTrend.select(
        'coefficients').arrayProject([0]).arrayFlatten([independents])  # Turn the array image into a multi-band image of coefficients
    new_coll = year_coll.map(lambda image: image.addBands(
        image.select(independents).multiply(harmonicTrendCoeffs).reduce('sum').rename('Fit'))).filter(  # Compute fitted values and add it to the original image
        ee.Filter.calendarRange(year, year, 'year'))
    # Merge the resulted image collection with the empty image collection
    fittedHarmonic = fittedHarmonic.merge(new_coll)

### 6. Generate random points within the area of interest
Using GEE doesn't allow us to generate random points as we wish (because the builtin function there doesn't allow us to controle the min distance between the points). In order to create random points with zero chance of having more than one point over the same pixel, we create an image of longitude & latitude bands and turn the pixels values of these two bands into points feature collection. After that we can select randomely from the points feature collection.

In [7]:
# Select number of points from the feature collection that we will sample
no_points = ee.Number(5)

# 'pixelLonLat' method Creates an image with two bands: 'longitude' and 'latitude', one stores longitude & the second stores latitude at each pixel in degrees.
rand_points = ee.Image.pixelLonLat().addBands(
    harmonic_modis.first()).reduceRegion(ee.Reducer.toList(), poly, 250)  # Reduce the 'longitude' and 'latitude' bands into a list
lat = ee.List(rand_points.get('latitude'))  # Get latitiude
lon = ee.List(rand_points.get('longitude'))  # Get longitude
lat_lon = lon.zip(lat) # Concat lon & lat into list to turn them into feature collection


def lat_lon_rand(ind):  # function that will turn Lat Long list into a point feature collection
    return ee.Feature(ee.Geometry.Point(ind))


# Applying 'lat_lon_rand' function
lat_lon_fc = ee.FeatureCollection(lat_lon.map(lat_lon_rand))

# Here we create a new attripute column with floating point numbers in the range 0.0 to 1.0. and Select randomely a 
# number of points (depending on no_points variable which is specificed in block 3)
rand_points = lat_lon_fc.randomColumn('Rand', 6).sort('Rand').limit(no_points)

### 7. Sample the random points with NDVI, EVI, & the fitted NDVI bands of the image collection
In this code block we sample the 'fittedHarmonic' image collection randomely generated points features. we use 'ee.Reducer.toList' function in with 'reduceRegion' which creates a reducer that collects its inputs into a list. Because we have two collections (Image collection & feature collection), we create a nested loop where we loop first over each point feature and then we loop over the image collection. Before we preform the sampling, we added the overall (from 2002 to 2013) mean values of our bands of interest('EVI_Mean', 'Fit_Mean', 'NDVI_Mean').

In [8]:
# Number of images in the image collection
img_coll_list = fittedHarmonic.toList(fittedHarmonic.size())

# Turn points feature collection into list (so that we can use it inside python For loop)
feat_coll_list = rand_points.toList(rand_points.size())
indic_means = fittedHarmonic.mean().select(['EVI', 'Fit', 'NDVI']).rename(
    ['EVI_Mean', 'Fit_Mean', 'NDVI_Mean'])  # reducing the image collection using mean function
df_list = ee.List([])  # empty list that will be filled with sampled values

for ind in range(feat_coll_list.size().getInfo()):  # looping over number of random points
    feat = ee.Feature(feat_coll_list.get(ind)).geometry()
    point_df = ee.List([])
    # looping over each image in the image collection
    for img in range(img_coll_list.size().getInfo()):
        img = ee.Image(img_coll_list.get(img))
        point_dic = img.select(['NDVI', 'EVI', 'Fit']).addBands(indic_means).reduceRegion(
            ee.Reducer.toList(numOptional=1), feat)
        point_df = point_df.add(ee.Dictionary(point_dic).values().flatten())
    df_list = df_list.add(point_df)

### 8. Convert sampled points values into 3D array (the values are python numpy array now)
The 3D array is as the following:


> The 0 axis we have images (253)

> The 1 axis we have bands (6) with this arrangment(EVI, EVI_Mean, Fit, Fit_Mean, NDVI, NDVI_Mean)

> The 2 axis we have random points (5 so far)


In [9]:
entire_val = df_list.getInfo()  # Retrive sampled image collection values
x_arr = fittedHarmonic.size().getInfo()  # Number of images on the x axis
y_arr = 6  # Number of bands on the y axis
z_arr = len(entire_val)  # Number of points sampled on the z axis
# creating array with diamntions equal to the variable we create earlier
empty_num = np.empty((x_arr, y_arr, z_arr))

for z in range(z_arr):  # Fill the array with sampled values
    for x in range(x_arr):
        empty_num[x, :, z] = np.array(entire_val[z][x])

### 9. Retrieve dates for each image and convert them into a datetime list 

In [10]:
def date_mil(img):  # Retrive date property of each image
    return ee.Date(ee.Image(img).get('system:time_start')).format('dd/MM/y')


img_coll_date = img_coll_list.map(date_mil).getInfo()
# Turn each element in the list into python datetime object
list_dates = [datetime.strptime(x, '%d/%m/%Y') for x in img_coll_date]

# Part 2: Generate charts and subplots


### 1. Retive random points coordinates
In this block we retrive the coordinates of the random points and round the values into 2 decimal digits.

In [11]:
coord_list = lat_lon.getInfo()
coord_list_short = []
for coor in range(len(coord_list)):
    lat = round(coord_list[coor][0], 2)
    lon = round(coord_list[coor][1], 2)
    coord_list_short.append([lat, lon])

### 2. Plot NDVI , NDVI mean & Harmonic Fit values annually
In the next two code block we plot the NDVI, NDVI mean, & Fit values annually using plotly library. We automate the plot using nested for loop. Since each subplot has a unique column/row index (grid like 4 by 3), we prepare these column/row values before hand to use it inside the loops.

In [12]:
# Preparing a 4x3 grid of subplots
year_labels = list(range(2001, 2014))  # List of years
columns = list(range(1, 4))  # Subplots columns
for i in range(3):  # list of repeated columns index according to the number of years to plot (12 years on 4 columns)
    columns = columns + list(range(1, 4))
# list of repeated rows index according to the number of years to plot (12 years on 3 rows)
rows = list(itertools.chain.from_iterable(itertools.repeat(i, 3)
                                          for i in list(range(1, 5))))

In [13]:
# Loop for annual plots 
for point in range(z_arr):
    plot_date = pd.DataFrame(list(zip(
        list_dates, empty_num[:, 2, point], empty_num[:, 4, point], empty_num[:, 5, point])), columns=['Dates', 'Fit', 'NDVI', 'NDVI_Mean'])
    plot_date['Year'] = plot_date['Dates'].map(lambda x: x.strftime('%Y'))
    uniq_years = plot_date['Year'].unique()
    legend = [False]*(len(uniq_years)-2)+[True]
    fig = make_subplots(
        rows=4, cols=3,
        subplot_titles=(year_labels[1:]),
        x_title='Month',
        y_title='NDVI Values'
    )

    fig.update_layout(title=f'NDVI & Harmonic Fit Point {str(point + 1)} ({str(coord_list_short[point])[1:-1]})')

    for i in range(1, 12):
        year_df_des = plot_date[plot_date['Year'] == uniq_years[i]]
        year_df_befo = plot_date[(plot_date['Dates'] >= f'{uniq_years[i-1]}-10-01') & (plot_date['Dates'] <= f'{uniq_years[i-1]}-12-31')]
        year_df = year_df_befo.append(year_df_des)
        # Gen title for overall graphic, and adjust
        fig.layout.paper_bgcolor = "white"
        fig.layout.plot_bgcolor = "white"
        fig.update_xaxes(tickformat='%d %b', showline=True,
                         linecolor="#444", showgrid=False, type='date')
        fig.update_yaxes(range=[0, 1], tick0=0.2, dtick=0.2, autorange=False,
                         showline=True, linecolor="#444", showgrid=False)
        fig.add_trace(go.Scatter(x=year_df['Dates'],
                                 y=year_df['Fit'],
                                 name="Harmonic Fit",
                                 line_color='red',
                                 opacity=0.8,
                                 showlegend=legend[i]),
                      row=rows[i-1], col=columns[i-1])
        fig.add_trace(go.Scatter(x=year_df['Dates'],
                                 y=year_df['NDVI'],
                                 name="NDVI",
                                 line_color='green',
                                 mode='markers',
                                 opacity=0.9,
                                 showlegend=legend[i]),
                      row=rows[i-1], col=columns[i-1])
        fig.add_trace(go.Scatter(x=year_df['Dates'],
                                 y=year_df['NDVI_Mean'],
                                 name="NDVI_Mean",
                                 opacity=0.9,
                                 line=dict(color='blue', width=3, dash='dash'),
                                 showlegend=legend[i]),
                      row=rows[i-1], col=columns[i-1])
        # Add shape regions
        fig.add_shape(go.layout.Shape(  # highlight during Oct - Dec
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            x0=datetime(int(uniq_years[i-1]), 10, 1),
            y0=0,
            x1=datetime(int(uniq_years[i]), 2, 28),
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0),
            row=rows[i-1], col=columns[i-1])

    # print the graphics
    fig.show()

### 3. Plot the random points with background from Sentinel 2 true color imagery 

In [14]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-4.22, 38.04], zoom_start=14, height=1200)

# Sentinel 2 image covering study area
sent_2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(poly).filter(
    ee.Filter.calendarRange(2019, 2019, 'year')).sort('CLOUDY_PIXEL_PERCENTAGE', True).first()
imageVisParam = {"opacity": 1, "bands": [
    "B4", "B3", "B2"], "min": 347.6, "max": 2480.4, "gamma": 1}

# Add Sentinel 2 image to the map object.
my_map.add_ee_layer(ee.Image(sent_2), imageVisParam, 'Sent_2')

# Add study area to the map object.
folium.GeoJson(
    poly.toGeoJSON(),
    name='Study_Area'
).add_to(my_map)

# Add random points to the map object.
for i in range(5):
    latit = ee.Feature(feat_coll_list.get(i)).getInfo()[
        'geometry']['coordinates'][0]
    longit = ee.Feature(feat_coll_list.get(i)).getInfo()[
        'geometry']['coordinates'][1]
    folium.Marker(
        location=[longit, latit],
        popup=f'Point_{i+1}'
    ).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Part 3: Generate summary statistics over the growing season 
2003 should reflect the growing season from November 2002 - March 2003, for example. The three summary statistics we'd like are area under the curve, median, and mean for each point. We will then convert these figures into one value for each year and export the table as a csv file for downloading. 

In [15]:
points_mean_df = pd.DataFrame(list(zip(list_dates, np.nanmean(empty_num, axis=(2))[:, 0], np.nanmean(
    empty_num, axis=(2))[:, 2], np.nanmean(empty_num, axis=(2))[:, 4])), columns=['Dates', 'EVI',  'Fit', 'NDVI'])
grow_seas_df = pd.DataFrame()

for seas in range(2002, 2014):
    season = f'{seas-1}-{seas}'
    grow_seas = pd.DataFrame(points_mean_df[(points_mean_df['Dates'] >= f'{seas-1}-11-01') & (points_mean_df['Dates'] <= f'{seas}-03-31')])
    evi_mean = round(grow_seas['EVI'].mean(), 3)
    ndvi_mean = round(grow_seas['NDVI'].mean(), 3)
    fit_mean = round(grow_seas['Fit'].mean(), 3)
    evi_med = round(grow_seas['EVI'].median(), 3)
    ndvi_med = round(grow_seas['NDVI'].median(), 3)
    fit_med = round(grow_seas['Fit'].median(), 3)
    evi_auc = round(np.trapz(grow_seas['EVI']), 3)
    ndvi_auc = round(np.trapz(grow_seas['NDVI']), 3)
    fit_auc = round(np.trapz(grow_seas['Fit']), 3)
    grow_stats = [season, evi_mean, evi_med, evi_auc, fit_mean,
                  fit_med, fit_auc, ndvi_mean, ndvi_med, ndvi_auc]
    grow_seas_df = pd.concat(
        [grow_seas_df, pd.DataFrame(grow_stats).T], axis=0, sort=False)
grow_seas_df.columns = ['Season', 'EVI_Mean', 'EVI_Median', 'EVI_AUC',
                        'Fit_Mean', 'Fit_Median', 'Fit_AUC', 'NDVI_Mean', 'NDVI_Median', 'NDVI_AUC']
print(grow_seas_df)
#grow_seas_df.to_csv('Stats_summary', index=False)

      Season EVI_Mean EVI_Median EVI_AUC Fit_Mean Fit_Median Fit_AUC  \
0  2001-2002     0.42      0.445   3.842     0.64      0.648   5.858   
0  2002-2003    0.488      0.485   4.442    0.637      0.632   5.803   
0  2003-2004    0.394       0.41   3.624    0.668      0.691   6.088   
0  2004-2005    0.482      0.532   3.863     0.64      0.624   5.144   
0  2005-2006    0.346      0.353   3.081    0.599      0.608   5.374   
0  2006-2007     0.56      0.558   5.052    0.669      0.664   6.007   
0  2007-2008    0.476      0.484   4.277     0.71      0.716   6.412   
0  2008-2009    0.401      0.383   3.175    0.601      0.614   4.819   
0  2009-2010    0.428      0.407   4.012    0.608       0.61   5.566   
0  2010-2011    0.415      0.397   3.881    0.635      0.605   5.822   
0  2011-2012    0.486      0.478   4.381    0.618      0.587   5.637   
0  2012-2013    0.435      0.388   3.491    0.635      0.635   5.114   

  NDVI_Mean NDVI_Median NDVI_AUC  
0     0.632       0.679    5